***
LIBRARIES: most required ones
***

In [22]:
%pip install torch torchvision torchsummary

Note: you may need to restart the kernel to use updated packages.


In [23]:
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # plot


import torch # pytorch
import torch.nn as nn # neural network
import torch.optim as optim # optimizer
from torchsummary import summary # model summary (architecture)
from torch.utils.data import DataLoader, random_split # dataloader, dataset split
from torchvision import transforms,datasets # image processing, datasets
import torchvision.models as models # pretrained models

DATA_PATH = './dataset'

***
DATA LOADING
***

In [69]:
#Transformation for image
transform_ori = transforms.Compose([transforms.RandomResizedCrop(128),   #create 64x64 image
                                    transforms.RandomHorizontalFlip(),    #flipping the image horizontally
                                    transforms.ToTensor(),                 #convert the image to a Tensor
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])  #normalize the image
                                    

In [70]:
dataset = datasets.ImageFolder(root = DATA_PATH, transform=transform_ori)
data_size = len(dataset)

In [71]:
test_size = int(data_size * 0.25)
train_size = int(data_size * 0.75)
generator = torch.Generator().manual_seed(42)
training, testing = random_split(dataset, [train_size, test_size], generator=generator)

In [77]:
#Make the dataset iterable
batch_size = 100
train_load = DataLoader(dataset = training, 
                                         batch_size = batch_size,
                                         shuffle = True) 

test_load = torch.utils.data.DataLoader(dataset = testing, 
                                         batch_size = batch_size,
                                         shuffle = False)

***
ARCHITECTURE DEFINING
***

In [106]:
# Create a neural net class
class Net(nn.Module):
    
    
    # Defining the Constructor
    def __init__(self, num_classes=5):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=1, padding=2)
        # self.block2 = self.conv_block(c_in=256, c_out=128, dropout=0.1, kernel_size=3, stride=1, padding=1)
        # self.block3 = self.conv_block(c_in=128, c_out=64, dropout=0.1, kernel_size=3, stride=1, padding=1)
        self.lastcnn = nn.Linear(in_features=32, out_features=5)        
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x) 
        x = torch.flatten(x)      
        x = self.lastcnn(x)        
        return torch.log_softmax(x, dim=1)
    
    # def conv_block(self, c_in, c_out, dropout, **kwargs):
    #     seq_block = nn.Sequential(
    #         nn.Conv2d(in_channels=c_in, out_channels=c_out, **kwargs),
    #         nn.BatchNorm2d(num_features=c_out),
    #         nn.ReLU(),
    #         nn.Dropout2d(p=dropout)
    #     )        
    #     return seq_block
    
device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"

# Create an instance of the model class and allocate it to the device
model = Net(num_classes=5).to(device)

print(model)

Net(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (lastcnn): Linear(in_features=32, out_features=5, bias=True)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


***
OBJECTS: CRITERION, OPTIMIZER, SCHEDULER(maybe)
***

In [97]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

***
TRAINING AND VALIDATION: Validation not neccessary but recommended
***

In [98]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)        
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)    
    acc = torch.round(acc * 100)    
    return acc

In [99]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [100]:
print("Begin training.")
for e in tqdm(range(1, 11)):    # TRAINING    train_epoch_loss = 0
    train_epoch_loss =0
    train_epoch_acc = 0    
    
    model.train()
    for X_train_batch, y_train_batch in train_load:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)        
        optimizer.zero_grad()        
        y_train_pred = model(X_train_batch).squeeze()        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)        
        train_loss.backward()
        optimizer.step()        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
    # # VALIDATION
    # with torch.no_grad():
    #     model.eval()
    #     val_epoch_loss = 0
    #     val_epoch_acc = 0
    #     for X_val_batch, y_val_batch in val_loader:
    #         X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)            
    #         y_val_pred = model(X_val_batch).squeeze()            
    #         y_val_pred = torch.unsqueeze(y_val_pred, 0)            
    #         val_loss = criterion(y_val_pred, y_val_batch)
    #         val_acc = multi_acc(y_val_pred, y_val_batch)            
    #         val_epoch_loss += train_loss.item()
    #         val_epoch_acc += train_acc.item()    
    #         loss_stats['train'].append(train_epoch_loss/len(train_loader))
    # loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_load))
    # accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
    print(f'Epoch {e+0:02}: | Train Loss: {train_epoch_loss/len(train_load):.5f} | Train Acc: {train_epoch_acc/len(train_load):.3f}')

Begin training.


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:09<?, ?it/s]


KeyboardInterrupt: 

***
PLOTTING: Loss & Accuracy
***

In [ ]:
# Methods you may need
# plt.figure, plt.subplot, plt.plot, plt.xlabel, plt.ylabel, plt.title, plt.legend, plt.show

***
SAVING
***

In [ ]:
width,height = 300,400
x = torch.Tensor(1, 3, width, height)
with torch.no_grad():
    traced_cell = torch.jit.trace(model, (x))
model_name = "./models/Y_model_1.pt" # CHANGE THIS TO YOUR OWN PATH (most improtatnt step)
torch.jit.save(traced_cell, model_name)


# torch.save () this is not allowed because it requires class definition before loading for the 
# model evaluation

NameError: name 'F' is not defined